In [ ]:
# 지금까지 가중치 매개변수 W와 b에 대한 손실함수의 기울기를 구하는 방법으로 '수치 미분'을 배웠다.
# '수치 미분'은 단순하고 구현하기 쉽지만, 계산 시간이 오래 걸린다는 단점이 있다.
# 따라서 '수치 미분'보다 구현하기는 어렵지만, 계산이 더 효율적인 '오차역전파법backpropagation'을  배워보기로 한다.

# 오차역전파법은 두 가지 방식으로 이해할 수 있다.
'''
1. 수식을 통해 이해 - 일반적인 방법
2. 계산 그래프를 통해 이해 - 시각적인 방법
'''

In [ ]:
# 계산 그래프computational graph는 계산 과정을 그래프로 나타낸 것이다.
# *그래프 : 복수의 노드node와 에지edge로 표현되는 자료구조

# 계산 그래프의 출발점으로부터 종착점으로 전파한다면 순전파forward propagation라고 하고, 반대로 종착점에서 출발점으로 전파한다면 역전파backword propagation이라고 한다.

# 계산 그래프의 이점 1
# '국소적 계산'을 전파함으로써 최종 결과를 얻을 수 있다
'''
국소적 계산은 전체에서 어떤 일이 벌어지든 상관없이, 자신과 관계된 정보만으로 결과를 출력할 수 있다는 점이다.
예를 들어, 슈퍼마켓에서 여러 식품을 구입할 경우, 모든 계산은 결국 (지금까지의 계산총액) + (추가적으로 살 물건 금액)이라는 두 피연산자와 하나의 연산자의 조합으로 분리해낼 수 있다.
이처럼 복잡한 계산을 각 노드에서 국소적 계산의 조합으로 분리해낼 수 있다.
'''

# 계산 그래프의 이점 2
# 역전파를 통해 미분을 효율적으로 계산할 수 있다.
'''
사과 가격이 오르면, 사과 갯수와 소비세를 모두 고려한 가격이 얼마나 오르는지 알고싶다고 가정하자.
이는 '사과 가격에 대한 지불 금액의 미분'을 구하는 문제이다. 이 미분값은 사과 값이 '아주 조금' 올랐을 때 지불 금액이 얼마나 증가하느냐를 표시하는 것이다.
역전파를 통하면 중간 과정과 최종적인 미분값들을 효과적으로 구할 수 있어, 다수의 미분을 효율적으로 구할 수 있다.
'''

In [ ]:
# 계산 그래프의 역전파
# 뒤쪽 노드에서 국소적 미분을 곱한 후 앞쪽 노드로 전달한다. (순전파의 역계산)

# 연쇄법칙chain rule
# 합성 함수(여러 함수로 구성된 함수)의 미분에 대한 성질이며, 다음과 같이 정의된다.
'합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.'
# 즉, 합성 함수 전체의 미분 값은 각 구성함수를 미분하여 그 값들을 곱하면 구할 수 있다. 역전파에서도 국소적 계산(미분)이 성립하는 것이다.

# 덧셈 노드의 역전파
# 덧셈 노드의 국소적 미분값은 1이므로 상류(그래프의 오른쪽)에서 하류(그래프의 왼쪽)으로 그대로 보내주면 된다.

# 곱셈 노드의 역전파
# z = xy의 식에서 국소적 미분값은 x에 대한 편미분:y, y에대한 편미분:x 이므로, 순전파 때 곱한 값과 '서로 바꾼값'을 곱해주는것이 역전파가 된다.

In [4]:
# 사과 쇼핑 구현

# 곱셈 노드
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    # 순전파
    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    # 역전파
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        
        return dx, dy

In [6]:
apple = 100
apple_num = 2
tax = 1.1 # 소비세

# 계층들
mul_apple_layer = MulLayer() # 사과 갯수 곱하는 층
mul_tax_layer = MulLayer() # 소비세 곱하는 층

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num) # 사과 * 갯수
price = mul_tax_layer.forward(apple_price, tax) # (사과 * 갯수) * 소비세 = 총 가격

print(price)

# 역전파
dprice = 1 # 상류(오른쪽)에서 온 값
dapple_price, dtax = mul_tax_layer.backward(dprice) # 각각 (사과값*사과갯수)와 소비세에 대한 곱셈노드 국소적 미분
dapple, dapple_num = mul_apple_layer.backward(dapple_price) # 사과 값과 사과 갯수에 대한 곱셈노드 국소적 미분

print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


In [8]:
# 덧셈 노드
class AddLayer:
    def __init__(self):
        pass # 초기화가 필요없다.
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1 # 덧셈노드에 대한 미분값은 항상 1이다
        dy = dout * 1
        return dx, dy

In [25]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer() # 사과 갯수 곱하는 층 ---> (1)
mul_orange_layer = MulLayer() # 오렌지 갯수 곱하는 층 ----> (2)
# (1)과 (2) 계층은 반드시 분리해서 선언해야한다.(클래스 내에 x 속성과 y 속성을 기억하고 있으므로)
add_fruits_layer= AddLayer() # 사과와 오렌지 더하는 층
mul_tax_layer = MulLayer() # 소비세 곱하는 층

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
fruits_price = add_fruits_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(fruits_price, tax)

print(price) # 최종 금액

# 역전파
dprice = 1
dfruits_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_fruits_layer.backward(dfruits_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

print(dapple, dapple_num, dorange, dorange_num, dtax)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650
